In [ ]:
from fastai.vision.all import *

In [ ]:
path = Path('/kaggle/input/mnist/mnist_png')

In [ ]:
path=untar_data(URLs.MNIST)

In [ ]:
training_path = path /'training'
print(training_path.ls())

In [ ]:
testing_path = path / 'testing/9'
print(testing_path.ls())

In [ ]:
dls = ImageDataLoaders.from_folder(
    path,  
    valid_pct=0.2, 
    seed=42,  
    item_tfms=Resize(192),  
    batch_tfms=aug_transforms()
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
x_batch, y_batch = dls.one_batch()
lambda_ = np.random.uniform(0.2, 0.8)  
mixup_images = lambda_ * x_batch[0] + (1 - lambda_) * x_batch[1] 
num_classes = 10 
labels_one_hot = F.one_hot(y_batch, num_classes=num_classes).float()
smooth_factor = 0.1
smooth_labels = labels_one_hot * (1 - smooth_factor) + (smooth_factor / num_classes)
mixup_labels = lambda_ * smooth_labels[0] + (1 - lambda_) * smooth_labels[1]
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(x_batch[0].cpu().permute(1, 2, 0))
axes[0].set_title("Original Image 1")
axes[0].axis('off')
axes[1].imshow(x_batch[1].cpu().permute(1, 2, 0))
axes[1].set_title("Original Image 2")
axes[1].axis('off')
axes[2].imshow(mixup_images.cpu().permute(1, 2, 0))
axes[2].set_title(f"MixUp (lambda={lambda_:.2f})")
axes[2].axis('off')
plt.show()
print(f"Original Label 1: {y_batch[0]} -> Smoothed Label 1: {smooth_labels[0]}")
print(f"Original Label 2: {y_batch[1]} -> Smoothed Label 2: {smooth_labels[1]}")
print(f"Mixed Label (lambda={lambda_:.2f}): {mixup_labels}")


In [ ]:
learn = vision_learner(
    dls, 
    resnet18,  
    metrics=accuracy)

In [ ]:
learn.lr_find(suggest_funcs=(minimum, steep))

In [ ]:
class MixUp(MixHandler):
    def __init__(self, 
        alpha:float=.4
    ): 
        super().__init__(alpha)
    def before_batch(self):
        lam = self.distrib.sample((self.y.size(0),)).squeeze().to(self.x.device)
        lam = torch.stack([lam, 1-lam], 1)
        self.lam = lam.max(1)[0]
        shuffle = torch.randperm(self.y.size(0)).to(self.x.device)
        xb1,self.yb1 = tuple(L(self.xb).itemgot(shuffle)),tuple(L(self.yb).itemgot(shuffle))
        nx_dims = len(self.x.size())
        self.learn.xb = tuple(L(xb1,self.xb).map_zip(torch.lerp,weight=unsqueeze(self.lam, n=nx_dims-1)))

        if not self.stack_y:
            ny_dims = len(self.y.size())
            self.learn.yb = tuple(L(self.yb1,self.yb).map_zip(torch.lerp,weight=unsqueeze(self.lam, n=ny_dims-1)))

In [ ]:
learn = vision_learner(
    dls, 
    resnet18,  
    metrics=accuracy,
    loss_func=CrossEntropyLossFlat(label_smoothing=0.1),  # label smoothing
    cbs=MixUp()  )

In [ ]:
def fit_one_cycle(self:Learner, n_epoch, lr_max=None, div=25., div_final=1e5, pct_start=0.25, wd=None,
                  moms=None, cbs=None, reset_opt=False, start_epoch=0):
    if self.opt is None: self.create_opt()
    self.opt.set_hyper('lr', self.lr if lr_max is None else lr_max)
    lr_max = np.array([h['lr'] for h in self.opt.hypers])
    scheds = {'lr': combined_cos(pct_start, lr_max/div, lr_max, lr_max/div_final),
              'mom': combined_cos(pct_start, *(self.moms if moms is None else moms))}
    self.fit(n_epoch, cbs=ParamScheduler(scheds)+L(cbs), reset_opt=reset_opt, wd=wd, start_epoch=start_epoch)

In [ ]:
def fine_tune(self:Learner, epochs, base_lr=2e-3, freeze_epochs=1, lr_mult=100,
              pct_start=0.3, div=5.0, **kwargs):
    self.freeze()
    self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
    base_lr /= 2
    self.unfreeze()
    self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)

In [ ]:
learn.fine_tune(10)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(5,5))